In [94]:
import pandas as pd
import numpy as np
from datetime import datetime

In [97]:
data = pd.read_csv('../230331/230403/230404/csv/GM.csv', index_col = 'Date')

In [100]:
#1번 함수 생성
#매개변수 추가
def create_band (x, col, start, end):
    # 인덱스를 시계열로 변경
    x.index = pd.to_datetime(x.index)
    
    # start, end를 시계열로 변경
    start = datetime.strptime(start, '%Y%m%d').isoformat()
    end = datetime.strptime(end, '%Y%m%d').isoformat()
    
    # 데이터를 시작시간부터 종료시간까지 필터링
    x = x.loc[start:end]
    
    #결측치와 이상치 제거

    x = x.loc[~x.isin([np.nan, np.inf, -np.inf]).any(axis = 'columns'), [col]]
    # x = x[[col]]
    
    #이동평균선 생성
    x['Center'] = x[col].rolling(20).mean()
    
    #상단밴드, 하단밴드 생성
    
    x['ub'] = x['Center'] + (2 * x[col].rolling(20).std())
    x['lb'] = x['Center'] - (2 * x[col].rolling(20).std())
    return x

In [101]:
create_band(data, 'Close','20110101', '20150101')

,Close,Center,ub,lb
Date,,,,
2011-01-03,37.060001,NaN,NaN,NaN
2011-01-04,37.900002,NaN,NaN,NaN
2011-01-05,38.070000,NaN,NaN,NaN
2011-01-06,38.900002,NaN,NaN,NaN
2011-01-07,38.980000,NaN,NaN,NaN
...,...,...,...,...
2014-12-24,33.430000,32.5620,34.464492,30.659507
2014-12-26,33.730000,32.6450,34.601199,30.688801
2014-12-29,34.599998,32.7035,34.821803,30.585196


In [77]:
#2번 함수 생성
def add_trade (x):
    # 기준이 되는 컬럼이 무엇인가?
    # 기준이 되는 컬럼은 컬럼 중 첫번째니까 df.columns[0]
    col = x.columns[0]    
    # trade 파생변수 생성
    x['trade'] = ''
    
    # 거래 내역 추가
    for i in x.index:
        if x.loc[i,col] > x.loc[i,'ub']:
            if x.shift(1).loc[i,'trade'] == 'buy':
                x.loc[i,'trade'] = ""
            else:
                x.loc[i,'trade'] = ""
        
        elif x.loc[i,col] < x.loc[i,'lb']:
            if x.shift(1).loc[i,'trade'] == 'buy':
                x.loc[i,'trade'] = 'buy'
            else:
                x.loc[i,'trade'] = 'buy'
        
        else:
            if x.shift(1).loc[i,'trade'] == 'buy':
                x.loc[i,'trade'] = 'buy'
            else:
                x.loc[i, 'trade'] = ''
    return x      
    print(x['trade'])

In [66]:
add_trade(data)['trade'].value_counts()

trade
buy    1130
       1032
Name: count, dtype: int64

In [83]:
data2 = create_band(data, 'Close')
data2 = add_trade(data2)

In [75]:
# 3번째 함수 생성

def add_rtn (x):
    col = x.columns[0]
    #수익률 파생변수 생성
    x['return'] = 1
    
    # 판매한 날의 수익률 대입
    
    rtn = 1.0
    buy = 0.0
    sell = 0.0
    
    for i in x.index :
        if (x.shift(1).loc[i, 'trade'] == '') & (x.loc[i,'trade'] == 'buy'):
            buy = x.loc[i, col]
            print('진입일:', i, '구매가격:', buy)
        
        elif (x.shift(1).loc[i, 'trade'] == 'buy') & (x.loc[i,'trade'] == ''):
            sell = x.loc[i,col]
            rtn = (sell - buy) / buy + 1
            x.loc[i,'return'] = rtn
            print('판매일:', i, '판매가격:', sell, '수익률:', rtn)
        
        if x.loc[i,'trade'] == '':
            buy = 0.0
            sell = 0.0
    
    acc_rtn = 1.0

    for i in x.index :
        rtn = x.loc[i,'return'] 
        acc_rtn *= rtn
        x.loc[i,'acc_rtn'] = acc_rtn
         
    print('누적수익률:', acc_rtn)
    return x            

In [102]:
add_rtn(data2)

진입일: 2011-02-02 구매가격: 35.68
판매일: 2011-04-05 판매가격: 32.869999 수익률: 0.9212443665919282
진입일: 2011-06-02 구매가격: 29.6
판매일: 2011-07-07 판매가격: 31.799999 수익률: 1.0743242905405406
진입일: 2011-07-27 구매가격: 28.139999
판매일: 2012-01-05 판매가격: 22.17 수익률: 0.7878465098737211
진입일: 2012-03-06 구매가격: 24.58
판매일: 2012-04-02 판매가격: 26.76 수익률: 1.0886899918633037
진입일: 2012-04-09 구매가격: 24.200001
판매일: 2012-08-07 판매가격: 20.42 수익률: 0.8438016180247265
진입일: 2013-02-20 구매가격: 27.1
판매일: 2013-03-20 판매가격: 29.200001 수익률: 1.0774908118081181
진입일: 2013-06-20 구매가격: 32.560001
판매일: 2013-09-10 판매가격: 37.0 수익률: 1.1363636014630343
진입일: 2013-10-03 구매가격: 35.290001
판매일: 2013-10-30 판매가격: 37.23 수익률: 1.0549730502982984
진입일: 2014-01-16 구매가격: 39.0
판매일: 2014-03-05 판매가격: 37.52 수익률: 0.9620512820512821
진입일: 2014-03-13 구매가격: 34.09
판매일: 2014-06-04 판매가격: 36.52 수익률: 1.0712819008506893
진입일: 2014-07-24 구매가격: 35.740002
판매일: 2014-11-17 판매가격: 32.310001 수익률: 0.9040290764393355
진입일: 2014-12-15 구매가격: 31.0
판매일: 2015-02-10 판매가격: 37.52 수익률: 1.2103225806451614
진입일: 2015

,Close,Center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-11-18,34.189999,NaN,NaN,NaN,,1.0,1.000000
2010-11-19,34.259998,NaN,NaN,NaN,,1.0,1.000000
2010-11-22,34.080002,NaN,NaN,NaN,,1.0,1.000000
2010-11-23,33.250000,NaN,NaN,NaN,,1.0,1.000000
2010-11-24,33.480000,NaN,NaN,NaN,,1.0,1.000000
...,...,...,...,...,...,...,...
2019-06-18,36.700001,35.45400,37.253548,33.654451,buy,1.0,1.148091
2019-06-19,36.779999,35.43650,37.173130,33.699870,buy,1.0,1.148091
2019-06-20,36.959999,35.50700,37.372713,33.641286,buy,1.0,1.148091


In [180]:
import bollinger as bg
import imp
imp.reload(bg)

<module 'bollinger' from 'c:\\Users\\user\\Desktop\\욱\\230413\\bollinger.py'>

In [105]:
df = pd.read_csv('../230331/230403/230404/csv/MSFT.csv', index_col = 'Date')

In [117]:
# 1번 함수 실행
data = bg.create_band(df, 'Adj Close', '20100101', '20200101')

In [118]:
data2 = bg.add_trade(data)

In [120]:
#3번 함수 실행
bg.add_rtn(data2)

진입일: 2010-01-22 00:00:00 구매가격: 22.948128
판매일: 2010-03-17 00:00:00 판매가격: 23.588829 수익률: 1.0279195322598862
진입일: 2010-05-06 00:00:00 구매가격: 23.07136
판매일: 2010-09-13 00:00:00 판매가격: 20.187721 수익률: 0.875012179602763
진입일: 2010-11-23 00:00:00 구매가격: 20.319853
판매일: 2011-01-06 00:00:00 판매가격: 23.312819 수익률: 1.1472926994107686
진입일: 2011-03-16 00:00:00 구매가격: 20.171436
판매일: 2011-04-06 00:00:00 판매가격: 21.278055 수익률: 1.0548606950937949
진입일: 2011-05-16 00:00:00 구매가격: 19.992422
판매일: 2011-06-27 00:00:00 판매가격: 20.639456 수익률: 1.032363962705469
진입일: 2011-08-04 00:00:00 구매가격: 21.245531
판매일: 2011-08-31 00:00:00 판매가격: 21.923595 수익률: 1.0319156061573607
진입일: 2011-11-01 00:00:00 구매가격: 21.420837
판매일: 2012-01-03 00:00:00 판매가격: 22.229851 수익률: 1.037767618510892
진입일: 2012-04-04 00:00:00 구매가격: 26.087456
판매일: 2012-06-19 00:00:00 판매가격: 25.829544 수익률: 0.9901135626256542
진입일: 2012-07-12 00:00:00 구매가격: 24.087934
판매일: 2012-09-06 00:00:00 판매가격: 26.551155 수익률: 1.1022595379080664
진입일: 2012-09-28 00:00:00 구매가격: 25.204538
판매일: 2013

,Adj Close,Center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-01-04,24.525019,24.056307,24.964478,23.148135,,1.0,1.000000
2010-01-05,24.532942,24.095134,25.015633,23.174635,,1.0,1.000000
2010-01-06,24.382378,24.133962,25.032815,23.235109,,1.0,1.000000
2010-01-07,24.128809,24.168828,25.004852,23.332803,,1.0,1.000000
2010-01-08,24.295214,24.206467,24.989876,23.423058,,1.0,1.000000
...,...,...,...,...,...,...,...
2019-06-18,135.160004,128.226002,136.343399,120.108604,,1.0,2.985058
2019-06-19,135.690002,128.665502,137.408349,119.922655,,1.0,2.985058
2019-06-20,136.949997,129.129502,138.604271,119.654733,,1.0,2.985058


In [181]:

data = pd.read_csv('../230331/230403/230404/csv/AAPL.csv', index_col = 'Date')

In [182]:
# 클래스 생성
class_a = bg.Invest(data, 'Adj Close')

In [183]:
class_a.col

'Adj Close'

In [184]:
class_a.create_band('20100101','20150101')

,Adj Close,Center,ub,lb
Date,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713
2010-01-05,26.829010,25.169503,27.288098,23.050908
2010-01-06,26.402260,25.307290,27.366449,23.248130
2010-01-07,26.353460,25.436879,27.410937,23.462821
2010-01-08,26.528664,25.525609,27.529742,23.521475
...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775
2014-12-26,105.449570,104.310338,109.569622,99.051054
2014-12-29,105.375557,104.078143,108.639948,99.516339


In [185]:
class_a.add_trade()

,Adj Close,Center,ub,lb,trade
Date,,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713,
2010-01-05,26.829010,25.169503,27.288098,23.050908,
2010-01-06,26.402260,25.307290,27.366449,23.248130,
2010-01-07,26.353460,25.436879,27.410937,23.462821,
2010-01-08,26.528664,25.525609,27.529742,23.521475,
...,...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775,buy
2014-12-26,105.449570,104.310338,109.569622,99.051054,buy
2014-12-29,105.375557,104.078143,108.639948,99.516339,buy


In [186]:
class_a.add_rtn()

진입일: 2010-01-22 00:00:00 구매가격: 24.747818
판매일: 2010-03-01 00:00:00 판매가격: 26.154476 수익률: 1.0568396777445188
진입일: 2010-08-24 00:00:00 구매가격: 30.026524
판매일: 2010-09-08 00:00:00 판매가격: 32.90366 수익률: 1.0958198158401553
진입일: 2011-03-16 00:00:00 구매가격: 41.299767
판매일: 2011-07-01 00:00:00 판매가격: 42.957966 수익률: 1.040150323366231
진입일: 2011-11-14 00:00:00 구매가격: 47.463268
판매일: 2011-12-27 00:00:00 판매가격: 50.876015 수익률: 1.0719029081604747
진입일: 2012-04-16 00:00:00 구매가격: 72.601524
판매일: 2012-06-18 00:00:00 판매가격: 73.308609 수익률: 1.0097392583659814
진입일: 2012-10-08 00:00:00 구매가격: 80.207954
판매일: 2013-07-29 00:00:00 판매가격: 57.243137 수익률: 0.7136840443530077
진입일: 2013-09-11 00:00:00 구매가격: 60.184383
판매일: 2013-10-18 00:00:00 판매가격: 65.48336 수익률: 1.0880457144505413
진입일: 2014-01-03 00:00:00 구매가격: 70.019096
판매일: 2014-03-25 00:00:00 판매가격: 70.960335 수익률: 1.0134426042861222
진입일: 2014-10-15 00:00:00 구매가격: 89.842468
판매일: 2014-10-23 00:00:00 판매가격: 96.557182 수익률: 1.0747387527243797
진입일: 2014-12-12 00:00:00 구매가격: 101.508751
누적수익률: 

,Adj Close,Center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713,,1.0,1.000000
2010-01-05,26.829010,25.169503,27.288098,23.050908,,1.0,1.000000
2010-01-06,26.402260,25.307290,27.366449,23.248130,,1.0,1.000000
2010-01-07,26.353460,25.436879,27.410937,23.462821,,1.0,1.000000
2010-01-08,26.528664,25.525609,27.529742,23.521475,,1.0,1.000000
...,...,...,...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775,buy,1.0,1.102717
2014-12-26,105.449570,104.310338,109.569622,99.051054,buy,1.0,1.102717
2014-12-29,105.375557,104.078143,108.639948,99.516339,buy,1.0,1.102717
